In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# SLAC XSIF to Bmad conversion

This requires code from LCLS_LATTICE

In [2]:
# Patch in the slac2bmad package
import sys, os
pypath = os.environ['LCLS_LATTICE']+'/bmad/conversion/python'
sys.path.append(pypath)

In [3]:
from slac2bmad.xsif import prepare_xsif, remove_comment_blocks, replace_set, replace_set_commands, fix_matrix, expand_names, fix_names, unfold_comments, fold_comments
from slac2bmad.desplit import desplit_eles, desplit_ele
from slac2bmad.replace import replace_element, replace_eles
from slac2bmad.bmad import finalize_bmad

from glob import glob
import shutil

import subprocess
import json
import os

# Custom element replacements (in Bmad)

In [4]:
NEWELES = {}

NEWELES['um10466'] = """
!------- Laser Heater Undulator for Copper Linac -------
my_lh_und_k = 0
um10466: wiggler, 
        type = "laser_heater_undulator",
        L_period = 0.054, 
        n_period = 10, 
        b_max = my_lh_und_k * 2*pi*m_electron / (c_light * 0.054), 
        L = 10*0.054 ! Was: 0.506263, 
        ds_step = 0.054
        
um10466[L] = um10466[L]/2 ! Will be doubled in desplitting process. 
!---------------------------------
    """
        
NEWELES['dh02b'] = """
! Shorten so that lh_und has an integer number of poles
dh02b: drift, l = 0.0846296264 - ( 10*0.054 - 0.506263 ) /2
"""

NEWELES['dh03a'] = """
! Shorten so that lh_und has an integer number of poles
dh03a: drift, l = 0.0845477112 - ( 10*0.054 - 0.506263 ) /2
"""    

In [5]:
# Add these repalcements
F2_LINAC_REPLACEMENTS = json.load(open('good_facet2_linac_replacements.json'))
for name, replace in F2_LINAC_REPLACEMENTS.items():
    NEWELES[name.lower()+'_full'] = replace   

In [6]:
def all_replacements(master_file):
    dat = {}
    dat.update(NEWELES)
    #if master_file.startswith('F2_'):
    #    print('F2replacements')
    #    dat.update(F2_NEWELES)
    #    return dat
    #else:
    #    raise 
    return dat
#all_replacements('F2_')        

# Convert all

In [7]:
!mkdir temp

mkdir: cannot create directory ‘temp’: File exists


In [8]:
# Clean
!rm *xsif *bmad *digested*

rm: cannot remove '*xsif': No such file or directory
rm: cannot remove '*bmad': No such file or directory
rm: cannot remove '*digested*': No such file or directory


In [9]:
!cp ~/repos/facet2-lattice/MAD/*.xsif .

In [10]:
XSIF_FILES=[f for f in os.listdir() if f.endswith('.xsif')]
for f in XSIF_FILES:
    prepare_xsif(f, save=False)

Preparing QF2Q.xsif
Preparing DL10.xsif
Preparing SCAV.xsif
Preparing BC20P.xsif
Preparing LI20.xsif
Preparing L3.xsif
Preparing QDDSQ.xsif
Preparing FACET2p_PRLTDR.xsif
Preparing L1.xsif
Preparing INJ.xsif
Preparing BD.xsif
Preparing BC11.xsif
Preparing L2.xsif
Preparing FACET2p_DRTBC11.xsif
Preparing F2_POSI.xsif
Preparing FACET2e_master.xsif
Preparing BA.xsif
Preparing FACET2p_master.xsif
Preparing BC20E.xsif
Preparing QFCQ.xsif
Preparing BC14.xsif
Preparing F2_ELEC.xsif
Preparing F2_SCAV.xsif
Preparing FACET2p_DR.xsif
Preparing F2_S10AIP.xsif
Preparing common.xsif


In [11]:
!mv *.xsif temp/

In [12]:
F2_MASTERS = [f for f in os.listdir('../../MAD') if f.startswith('F2_')]
F2_MASTERS

['F2_POSI.xsif', 'F2_ELEC.xsif', 'F2_SCAV.xsif', 'F2_S10AIP.xsif']

In [13]:
TEMPDIR = './temp/'
WORKDIR = './work/'

In [14]:
!mkdir {TEMPDIR}
!mkdir {WORKDIR}

mkdir: cannot create directory ‘./temp/’: File exists
mkdir: cannot create directory ‘./work/’: File exists


# Process 

In [15]:
DEST = os.path.expandvars('/home/mpe/repos/facet2-lattice/bmad/master/')
assert os.path.exists(DEST)

In [20]:
def process_master(master):
    
    print(f'Converting {master}')
    
    shutil.copytree(TEMPDIR, WORKDIR, dirs_exist_ok=True)
    !ls work
    # New method
    SCRIPT = f'cd ~/repos/facet2-lattice/bmad/conversion/work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f {master}'
    print(SCRIPT)
    res = subprocess.run(SCRIPT, shell=True, cwd=WORKDIR)
    
    assert res.returncode == 0
    
    BMAD_FILES=glob(WORKDIR+'/*bmad')

    REPLACEMENTS = all_replacements(master)
    #REPLACEMENTS={}

    for f in BMAD_FILES:       
        finalize_bmad(f, replacements=REPLACEMENTS, verbose=False)   
    
    print(f'    Copying all to {DEST}')
    for f in BMAD_FILES:
        #print(f'copying {f} to {DEST}')
        shutil.copy(f, DEST)
    
    
process_master('F2_ELEC.xsif')
#process_master('INJ.xsif')

Converting F2_ELEC.xsif
BA.xsif     common.bmad     F2_SCAV.xsif	  INJ.xsif   LI20.xsif
BC11.bmad   common.xsif     FACET2e_master.bmad   L1.bmad    QDDSQ.xsif
BC11.xsif   DL10.bmad	    FACET2e_master.xsif   L1.xsif    QF2Q.xsif
BC14.bmad   DL10.xsif	    FACET2p_DRTBC11.xsif  L2.bmad    QFCQ.xsif
BC14.xsif   F2_ELEC.bmad    FACET2p_DR.xsif	  L2.xsif    SCAV.bmad
BC20E.xsif  F2_ELEC.xsif    FACET2p_master.xsif   L3.bmad    SCAV.xsif
BC20P.xsif  F2_POSI.xsif    FACET2p_PRLTDR.xsif   L3.xsif
BD.xsif     F2_S10AIP.xsif  INJ.bmad		  LI20.bmad
cd ~/repos/facet2-lattice/bmad/conversion/work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_ELEC.xsif
Input lattice file is:  F2_ELEC.xsif
Output lattice file is: F2_ELEC.bmad
    Copying all to /home/mpe/repos/facet2-lattice/bmad/master/


In [21]:
all_replacements('F2_ELEC.xsif').keys()

dict_keys(['um10466', 'dh02b', 'dh03a', 'l0af_full', 'l0bf_full', 'k11_2a_full', 'l1xf_full', 'k11_4a_full', 'k11_6a_full', 'k12_2a_full', 'k12_3a_full', 'k12_4a_full', 'k12_5a_full', 'k12_6a_full', 'k12_7a_full', 'k12_8a_full', 'k12_8d_full', 'k13_2a_full', 'k13_3a_full', 'k13_4a_full', 'k13_5a_full', 'k13_6a_full', 'k13_7a_full', 'k13_8a_full', 'k13_8d_full', 'k14_2a_full', 'k14_3a_full', 'k14_4a_full', 'k14_5a_full', 'k14_6a_full', 'k14_6d_full', 'k15_1a_full', 'k15_2a_full', 'k15_3a_full', 'k15_4a_full', 'k15_5a_full', 'k15_6a_full', 'k15_7a_full', 'k15_8a_full', 'k15_8d_full', 'k16_2a_full', 'k16_3a_full', 'k16_4a_full', 'k16_5a_full', 'k16_6a_full', 'k16_7a_full', 'k16_8a_full', 'k16_8d_full', 'k17_2a_full', 'k17_3a_full', 'k17_4a_full', 'k17_5a_full', 'k17_6a_full', 'k17_7a_full', 'k17_8a_full', 'k17_8d_full', 'k18_2a_full', 'k18_3a_full', 'k18_4a_full', 'k18_5a_full', 'k18_6a_full', 'k18_7a_full', 'k18_8a_full', 'k18_8d_full', 'k19_2a_full', 'k19_3a_full', 'k19_4a_full', 'k19_4

In [22]:
for m in F2_MASTERS:
    process_master(m)

Converting F2_POSI.xsif
BA.xsif     common.bmad     F2_SCAV.xsif	  INJ.xsif   LI20.xsif
BC11.bmad   common.xsif     FACET2e_master.bmad   L1.bmad    QDDSQ.xsif
BC11.xsif   DL10.bmad	    FACET2e_master.xsif   L1.xsif    QF2Q.xsif
BC14.bmad   DL10.xsif	    FACET2p_DRTBC11.xsif  L2.bmad    QFCQ.xsif
BC14.xsif   F2_ELEC.bmad    FACET2p_DR.xsif	  L2.xsif    SCAV.bmad
BC20E.xsif  F2_ELEC.xsif    FACET2p_master.xsif   L3.bmad    SCAV.xsif
BC20P.xsif  F2_POSI.xsif    FACET2p_PRLTDR.xsif   L3.xsif
BD.xsif     F2_S10AIP.xsif  INJ.bmad		  LI20.bmad
cd ~/repos/facet2-lattice/bmad/conversion/work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_POSI.xsif
Input lattice file is:  F2_POSI.xsif
Output lattice file is: F2_POSI.bmad
    Copying all to /home/mpe/repos/facet2-lattice/bmad/master/
Converting F2_ELEC.xsif
BA.xsif     common.bmad   F2_S10AIP.xsif	FACET2p_PRLTDR.xsif  L3.xsif
BC11.bmad   common.xsif   F2_SCAV.xsif		INJ.bmad	     LI20.bmad
BC11.xsif   DL10.

# Final cleanup

In [19]:
!rm -r {TEMPDIR}
!rm -r {WORKDIR}